# Training on Mobility Data
In this notebook we will train models on the mobility data.

In [ ]:
import os
import pandas as pd
from google.colab import auth
from datetime import datetime
auth.authenticate_user()
!gcloud source repos clone github_aistream-peelout_flow-forecast --project=gmap-997
os.chdir('/content/github_aistream-peelout_flow-forecast')
!pip install -r requirements.txt
!mkdir data
from flood_forecast.trainer import train_function
!pip install git+https://github.com/CoronaWhy/task-geo.git
!pip install git+https://github.com/coronawhy/task-ts
import wandb
!python corona_train.py --config da_config.json 
!gsutil cp gs://task_ts_data/acs2017_county_data.csv .

def format_pd(file_path="../../input/us-census-demographic-data/acs2015_census_tract_data.csv", output_path=""):
    df = pd.read_csv(file_path)
    df["full_county"] = df["County"] + "_" + df["State"]
    df.dropna().to_csv(file_path)

# Get config files and weights for auto-encoder
!wget -O "weight.pth" https://storage.googleapis.com/coronaviruspublicdata/experiments/09_September_202006_03AM_model.pth
!wget -O "config_right.json" https://storage.googleapis.com/coronaviruspublicdata/experiments/09_September_202006_03AM.json
!gsutil cp gs://task_ts_data/config_right.json . 
format_pd("acs2017_county_data.csv")

In [ ]:
!gsutil cp gs://task_ts_data/config_right.json . 
!wget -O "core_weights.pth" https://storage.googleapis.com/coronaviruspublicdata/experiments/18_October_202006_28AM_model.pth

Copying gs://task_ts_data/config_right.json...
/ [1 files][  1.5 KiB/  1.5 KiB]                                                
Operation completed over 1 objects/1.5 KiB.                                      
--2020-12-20 19:51:55--  https://storage.googleapis.com/coronaviruspublicdata/experiments/18_October_202006_28AM_model.pth
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.195.128, 172.253.117.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128767 (126K) [application/octet-stream]
Saving to: ‘core_weights.pth’

core_weights.pth    100%[===================>] 125.75K  --.-KB/s    in 0.002s  

2020-12-20 19:51:55 (80.2 MB/s) - ‘core_weights.pth’ saved [128767/128767]



Now that we have all the basic things setup lets define our config file. Since mobility data has a longer lag we will need to define our parameter sweeps to have a longer lookback window.

In [ ]:
def make_config_file(file_path, df_len, weight_path="core_weights.pth"):
  run = wandb.init(project="covid_forecast", entity="covid")
  wandb_config = wandb.config
  train_number = df_len*.6
  validation_number = df_len *.9
  number = wandb_config["forecast_history"]-1
  config_default={                 
    "model_name": "DARNN",
    "model_type": "PyTorch",
    "model_params": {
        "n_time_series":9,
        "forecast_history":wandb_config["forecast_history"],
        "hidden_size_encoder":wandb_config["hidden_encoder"],
        "decoder_hidden_size": wandb_config["hidden_decoder"],
        "out_feats": 1,
        "dropout": wandb_config["dropout"],
        "gru_lstm": False,
        "meta_data":{
            "method":"Bilinear2",
            "meta_dim":128,
            "params":{
                "in1_features":8,
                "in2_features":128,
                "out_features":8
            },
            "da_method":"down_sample45"
            
        }
    },
    "dataset_params":
    {  "class": "default",
       "training_path": file_path,
       "validation_path": file_path,
       "test_path": file_path,
       "forecast_test_len":14,
       "batch_size":wandb_config["batch_size"],
       "forecast_history":wandb_config["forecast_history"]-1,
       "forecast_length":wandb_config["out_seq_length"],
       "train_end": int(train_number),
       "valid_start":int(train_number+1),
       "valid_end": int(validation_number),
       "test_start":int(train_number+1),
       "target_col": ["rolling_7"],
       "relevant_cols": ["rolling_7", "mobility_retail_recreation",	"mobility_grocery_pharmacy",	"mobility_parks",	"mobility_transit_stations",	"mobility_workplaces",	"mobility_residential"],
       "scaler": "StandardScaler", 
       "interpolate": False,
        "sort_column": "date",
     "feature_param":
     {
         "datetime_params":{
            "day_of_week":"numerical",
            "month": "numerical"
         }
     }
    },
    "training_params":
    {
       "criterion":"MASELoss",
       "optimizer": "Adam",
       "criterion_params":{
           "baseline_method":"mean"
       },
       "optim_params":
       {

       },
       "lr": wandb_config["lr"],
       "epochs": 10,
       "batch_size":wandb_config["batch_size"]
    
    },
    "meta_data": wandb_config["meta_data"],
    "GCS": True,
    "early_stopping":
    {
        "patience":3
    },
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["RMSE", "DilateLoss"],
   "inference_params":
   {     
         "datetime_start":"2020-09-19",
          "hours_to_forecast":16, 
          "num_prediction_samples": 100,
          "test_csv_path":file_path,
          "decoder_params":{
              "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1
          },
          "dataset_params":{
             "file_path": file_path,
             "forecast_history":wandb_config["forecast_history"]-1,
             "forecast_length":wandb_config["out_seq_length"],
             "relevant_cols": ["rolling_7", "mobility_retail_recreation",	"mobility_grocery_pharmacy",	"mobility_parks",	"mobility_transit_stations",	"mobility_workplaces",	"mobility_residential"],
             "target_col": ["rolling_7"],
             "scaling": "StandardScaler",
             "interpolate_param": False,
              "sort_column": "date",
      "feature_params":
     {
         "datetime_params":{
            "day_of_week":"numerical",
            "month": "numerical"
         }
     }
          }
      }
  }
  if weight_path: 
    config_default["weight_path"] = weight_path
  wandb.config.update(config_default)
  print(config_default)
  return config_default

wandb_sweep_config_full = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "batch_size": {
            "values": [10, 25, 30]
        },
        "lr":{
            "values":[0.001, 0.0001, .01, .1]
        },
        "forecast_history":{
            "values":[8]
        },
        "out_seq_length":{
            "values":[1]
        },
        "hidden_encoder":
        {
            "values":[32]
        },
        "hidden_decoder":
        {
            # 32, 64, 128 
            "values":[64]
        }, 
        "meta_data":{
            "values": [{
                "path":"config_right.json",
                "column_id": "full_county",
                "uuid": "Harris County_Texas",
                "dim":128,
                "input_method": "down_sample"
            }]
        },
        "dropout":
        {
            "values": [0.1, 0.5, 0.8]
        }
  
    }
}

In [ ]:
!wandb login
os.environ['MODEL_BUCKET'] = "coronaviruspublicdata"
os.environ["ENVIRONMENT_GCP"] = "Colab"
os.environ["GCP_PROJECT"] = "gmap-997"
# Flow Transfer Data 
!gsutil cp gs://predict_cfs/experiments/25_May_202010_29PM_model.pth .
# County Transfer Weights
!gsutil cp gs://coronaviruspublicdata/experiments/19_June_202011_00PM_model.pth .
# Flow + County Transfer Weights		
!gsutil cp gs://coronaviruspublicdata/experiments/22_June_202003_05PM_model.pth	.
!gsutil cp gs://coronaviruspublicdata/experiments/12_July_202011_43PM_model.pth .


wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Copying gs://predict_cfs/experiments/25_May_202010_29PM_model.pth...
/ [1 files][  4.7 MiB/  4.7 MiB]                                                
Operation completed over 1 objects/4.7 MiB.                                      
Copying gs://coronaviruspublicdata/experiments/19_June_202011_00PM_model.pth...
/ [1 files][  9.2 MiB/  9.2 MiB]                                                
Operation completed over 1 objects/9.2 MiB.                                      
Copying gs://coronaviruspublicdata/experiments/22_June_202003_05PM_model.pth...
/ [1 files][  9.2 MiB/  9.2 MiB]                                                
Operation completed over 1 objects/9.2 MiB.                                      
Copying gs://coronaviruspublicdata/experiments/12_July_202011_43PM_mode

In [ ]:
!gsutil cp gs://task_ts_data/12-13-20/data/United_States__Texas__Harris_County.csv	.
!gsutil cp gs://task_ts_data/12-13-20/data/United_States__Florida__Palm_Beach_County.csv .
!gsutil cp gs://task_ts_data/12-13-20/data/United_States__Massachusetts__Middlesex_County.csv .
!gsutil cp gs://task_ts_data/12-13-20/data/United_States__Kansas__Chase_County.csv .
!gustil cp gs://task_ts_data/12-13-20/United_States__Kansas__Chase_County.csv .
!gsutil cp gs://task_ts_data/12-13-20/United_States__New_York__Albany_County.csv .

Copying gs://task_ts_data/12-13-20/data/United_States__Texas__Harris_County.csv...
/ [1 files][ 40.5 KiB/ 40.5 KiB]                                                
Operation completed over 1 objects/40.5 KiB.                                     
Copying gs://task_ts_data/12-13-20/data/United_States__Florida__Palm_Beach_County.csv...
/ [1 files][ 42.6 KiB/ 42.6 KiB]                                                
Operation completed over 1 objects/42.6 KiB.                                     
Copying gs://task_ts_data/12-13-20/data/United_States__Massachusetts__Middlesex_County.csv...
/ [1 files][ 44.6 KiB/ 44.6 KiB]                                                
Operation completed over 1 objects/44.6 KiB.                                     
CommandException: No URLs matched: gs://task_ts_data/12-13-20/data/United_States__Kansas__Chase_County.csv
/bin/bash: gustil: command not found
Copying gs://task_ts_data/12-13-20/United_States__New_York__Albany_County.csv...
/ [1 files][ 21.6 Ki

In [ ]:
df =  pd.read_csv("United_States__Texas__Harris_County.csv")
len_df = len(df[8:])
df["new_cases"] = df["cases"].diff()
df["rolling_7"]=df["new_cases"].rolling(7, win_type='triang').mean()
df["datetime"] = df["date"]
df = df
df[8:].to_csv("United_States__Texas__Harris_County.csv")

In [ ]:
df["datetime"]

0      2020-03-21
1      2020-03-22
2      2020-03-23
3      2020-03-24
4      2020-03-25
          ...    
109    2020-07-08
110    2020-07-09
111    2020-07-10
112    2020-07-11
113    2020-07-12
Name: datetime, Length: 114, dtype: object

In [ ]:
sweep_id = wandb.sweep(wandb_sweep_config_full,'covid', 'covid_forecast')
wandb.agent(sweep_id, lambda: train_function("PyTorch", make_config_file("United_States__Texas__Harris_County.csv", len_df,  weight_path="core_weights.pth")))


Create sweep with ID: hl6zk3ef
Sweep URL: https://wandb.ai/covid/covid_forecast/sweeps/hl6zk3ef


wandb: Agent Starting Run: be110zod with config:
wandb: 	batch_size: 10
wandb: 	dropout: 0.1
wandb: 	forecast_history: 8
wandb: 	hidden_decoder: 64
wandb: 	hidden_encoder: 32
wandb: 	lr: 0.001
wandb: 	meta_data: {'column_id': 'full_county', 'dim': 128, 'input_method': 'down_sample', 'path': 'config_right.json', 'uuid': 'Middlesex County_Massachusetts'}
wandb: 	out_seq_length: 1
wandb: Currently logged in as: igodfried (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


{'model_name': 'DARNN', 'model_type': 'PyTorch', 'model_params': {'n_time_series': 9, 'forecast_history': 8, 'hidden_size_encoder': 32, 'decoder_hidden_size': 64, 'out_feats': 1, 'dropout': 0.1, 'gru_lstm': False, 'meta_data': {'method': 'Bilinear2', 'meta_dim': 128, 'params': {'in1_features': 8, 'in2_features': 128, 'out_features': 8}, 'da_method': 'down_sample45'}}, 'dataset_params': {'class': 'default', 'training_path': 'United_States__Texas__Harris_County.csv', 'validation_path': 'United_States__Texas__Harris_County.csv', 'test_path': 'United_States__Texas__Harris_County.csv', 'forecast_test_len': 14, 'batch_size': 10, 'forecast_history': 7, 'forecast_length': 1, 'train_end': 135, 'valid_start': 136, 'valid_end': 203, 'test_start': 136, 'target_col': ['rolling_7'], 'relevant_cols': ['rolling_7', 'mobility_retail_recreation', 'mobility_grocery_pharmacy', 'mobility_parks', 'mobility_transit_stations', 'mobility_workplaces', 'mobility_residential'], 'scaler': 'StandardScaler', 'interp


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below


...


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).


plotting with CI now


epoch,6
validation_loss,0.2594
unscaled_validation_loss,0.00897
_step,46
_runtime,16
_timestamp,1608494752
loss,0.5599
test_loss,0.51124
unscaled_test_loss,0.51124
average_prediction_sharpe,5e-05


epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
validation_loss,▁▃▃▃▄▇█
unscaled_validation_loss,▁▃▃▃▄▇█
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▁▁▁▁▁▂▂▂▃▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
_timestamp,▁▁▁▁▁▁▁▁▁▂▂▂▃▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
loss,█▆▄▃▃▂▁
test_loss,▁
unscaled_test_loss,▁
average_prediction_sharpe,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## Loading the Data
In this section we will load our data using the data crawler.


In [ ]:
# Historical code to delete later
df_list_full_all_counties_total = [make_df_from_csv("United_States__Florida__Palm_Beach_County.csv"), make_df_from_csv("United_States__Florida__Palm_Beach_County.csv"), make_df_from_csv("United_States__Massachusetts__Middlesex_County.csv"), make_df_from_csv("United_States__Texas__Harris_County.csv")]
#df_list = loop_through_locations(df, minimum_datapoints_threshold=80)
#important_cities_list2 = ["United_States__California__San_Diego_County", "United_States__Florida__Broward_County", "United_States__Florida__Palm_Beach_County"]

NameError: ignored

In [ ]:
import pandas as pd
def get_most_recent_file(file_path):
  list_of_files = glob.glob(file_path+"/*.pth") # * means all if need specific format then *.csv
  if len(list_of_files) > 1:
    latest_file = max(list_of_files, key=os.path.getctime)
    return latest_file
  return None

def format_corona_data(region_df:pd.DataFrame, region_name:str):
  """
  Format data for a specific region into 
  a format that can be used with flow forecast. 
  """
  if region_name == 'county':
    region_name = region_df['full_county'].iloc[0]
  elif region_name=='state':
    region_name = region_df['state'].iloc[0]
  #else:
    #region_name = region_df['country'].iloc[0]
  d = pd.to_datetime(region_df['date'])
  region_df['date'] = pd.to_datetime(region_df['date'])
  region_df['month']=d.map(lambda x: x.month)
  region_df['weekday'] = d.map(lambda x: x.weekday())
  region_df['datetime'] = region_df.date
  region_df.index = region_df.date
  region_df = region_df.sort_index()
  region_df = region_df.fillna(0)
  region_df['new_cases'] = region_df['cases'].diff()
  region_df.iloc[0]['new_cases'] = 0
  region_df["rolling_7"] = region_df["new_cases"].rolling(7, win_type='triang').sum()/7.0
  region_df["diff_rolling"] = region_df["rolling_7"].diff()
  region_df= region_df.fillna(0)
  region_df.to_csv(region_name+".csv")
  print(region_df.head(9))
  return region_df, len(region_df), region_name+".csv"

def run_full_geo_code(df_list, start_index, end_index, use_transfer=True):
  for i in range(start_index, end_index):
    file_path, len_df, file_path_name = format_corona_data(df_list[i], 'county')
    print("Length of data frame is: ")
    print(len_df)
    latest_file = get_most_recent_file("model_save")
    sweep_full = wandb.sweep(wandb_sweep_config_transfer, project="covid_forecast", entity='covid')
    wandb.agent(sweep_full, lambda:train_function("PyTorch", make_config_file(file_path_name, len_df)), core_weights.pth)

def loop_special_counties(df):
  df['date'] = pd.to_datetime(df['date'])
  file_path, len_df, file_path2 = format_corona_data(df, "county")
  sweep_id = wandb.sweep(wandb_sweep_config_full,'covid', 'covid_forecast')
  wandb.agent(sweep_id, lambda: train_function("PyTorch", make_config_file(file_path2, len_df,  weight_path="core_weights.pth")))

for df in df_list_full_all_counties_total[3:]:
  loop_special_counties(df)
  break
#important_cities_list = ["United_States__California__Los_Angeles_County", "United_States__Illinois__Cook_County", "United_States__Arizona__Maricopa_County", "United_States__Massachusetts__Middlesex_County", "United_States__Texas__Dallas_County", "United_States__Texas__Harris_County", "United_States__Florida__Miami_Dade_County", "United_States__California__Riverside_County", "United_States__Colorado__Denver_County", "United_States__Ohio__Cuyahoga_County", "United_States__New_York__Queens_County", "United_States__New_York__Bronx_County", "United_States__New_York__New_York_County", "United_States__New_York__Richmond_County", "United_States__Michigan__Wayne_County", "United_States__Missouri__St._Louis_County"]
#important_cities_list3 = ["United_States__Maine__Cumberland_County", "United_States__Maine__York_County", "United_States__California__Los_Angeles_County", "United_States__Maine__Aroostook_County", "United_States__Maine__Androscoggin_County", "United_States__Maine__Franklin_County"]

#print(important_cities_list[14:])

In [ ]:
!ls